In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import re
import os
from pathlib import Path

In [3]:
from torch.utils.tensorboard import SummaryWriter
from core.logger import parse_train_logs_metric, parse_val_logs_metric

In [4]:
exp_path = Path('/home/olivieri/exp/data/private/exps/seg/4-contrastive-B_PR_vs_dT/4.2-from_good_ckp/lambda0.05_251028_1212')
log_path = exp_path / 'logs' / '4.2-from_good_ckp-lambda0.05-251028_1212.log'

In [5]:
batch_xen_losses = parse_train_logs_metric(log_path, metric_name='batch_loss')
batch_aux_losses = parse_train_logs_metric(log_path, metric_name='batch_aux_loss')

In [6]:
num_steps_per_epoch = 46
loss_lambda = 0.05

In [7]:
# parse_val_logs_metric(filepath, 'val_mIoU');

In [8]:
exp_name = '4.2-from_good_ckp-lambda0.05-251028_1212'

tb_log_dir_path = exp_path / 'logs' / exp_name

In [9]:
os.makedirs(tb_log_dir_path, exist_ok=True)
tb_writer = SummaryWriter(log_dir=tb_log_dir_path)

In [10]:
for e, step_values in batch_xen_losses.items():
    for s in step_values.keys():
        xen_loss = batch_xen_losses[e][s]
        aux_loss = batch_aux_losses[e][s]
        tb_writer.add_scalar(f"train/aux_xen_ratio", aux_loss/xen_loss, (e-1)*num_steps_per_epoch + s)
        tb_writer.add_scalar(f"train/aux_xen_ratio_after_lambda", (aux_loss*loss_lambda)/(xen_loss*(1-loss_lambda)), (e-1)*num_steps_per_epoch + s)

In [11]:
tb_writer.close()